In [1]:
pip install opencv-python

In [2]:
import cv2
import numpy as np

### Création d'étiquette

In [3]:
def img_etiquette(nom_image):
    nom = nom_image.split('.')[-3] 
    if nom=="Oumaima":
         return np.array([1,0])
    elif nom=="Abderrahman":
         return np.array([0,1])  

### Création de données

In [4]:
import os
from random import shuffle
from tqdm import tqdm

In [5]:
def Data():
    data = []
    for img in tqdm(os.listdir("Images")):
        chemin=os.path.join("Images",img)
        img_data = cv2.imread(chemin, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        data.append([np.array(img_data), img_etiquette(img)])
    shuffle(data)  
    return data

In [6]:
data = Data()

100%|████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:01<00:00, 1098.65it/s]


In [7]:
train = data[:1400]  
test = data[1400:]
x_train = np.array([i[0] for i in train]).reshape(-1,50,50,1)
print(x_train.shape)
y_train = [i[1] for i in train]
x_test = np.array([i[0] for i in test]).reshape(-1,50,50,1)
print(x_test.shape)
y_test = [i[1] for i in test]

(1400, 50, 50, 1)
(600, 50, 50, 1)


### Création du model

In [8]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install tflearn

Note: you may need to restart the kernel to use updated packages.


In [10]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
tf.compat.v1.reset_default_graph()
convnet = input_data(shape=[50,50,1])
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate = 0.001,loss='categorical_crossentropy')
model = tflearn.DNN(convnet, tensorboard_verbose=1)
model.fit(x_train, y_train, n_epoch=30, validation_set=(x_test, y_test),show_metric=True,run_id="FRS")

Training Step: 659  | total loss: 0.00001 | time: 2.963s
| Adam | epoch: 030 | loss: 0.00001 - acc: 1.0000 -- iter: 1344/1400
Training Step: 660  | total loss: 0.00001 | time: 4.114s
| Adam | epoch: 030 | loss: 0.00001 - acc: 1.0000 | val_loss: 0.00000 - val_acc: 1.0000 -- iter: 1400/1400
--


### Pour visualisation des données et prediction

In [13]:
def Visualisation_data():
    Vdata = []
    for img in tqdm(os.listdir("Vimages")):
        chemin = os.path.join("Vimages", img)
        img_num = img.split('.')[0] 
        img_data = cv2.imread(chemin, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        Vdata.append([np.array(img_data), img_num])
    shuffle(Vdata)
    return Vdata

In [14]:
Vdata = Visualisation_data()

100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 864.35it/s]


import matplotlib.pyplot as plt  

fig = plt.figure(figsize=(20,20))
for num, data in enumerate(Vdata[:12]):
    img_data = data[0]
    y = fig.add_subplot(4,4, num+1)
    image = img_data
    data = img_data.reshape(50,50,1)
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 0:
        img_etiquette = 'Oumaima'
    else:
        img_etiquette = 'Abderrahman'
        
    y.imshow(image, cmap='gray')
    plt.title(img_etiquette)
    
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()